# مکانیزم‌های توجه و مدل‌های ترانسفورمر

یکی از مشکلات اصلی شبکه‌های بازگشتی این است که تمام کلمات در یک دنباله تأثیر یکسانی بر نتیجه دارند. این موضوع باعث عملکرد زیر حد مطلوب در مدل‌های استاندارد LSTM برای وظایف دنباله به دنباله، مانند شناسایی موجودیت‌های نام‌دار و ترجمه ماشینی می‌شود. در واقعیت، کلمات خاصی در دنباله ورودی معمولاً تأثیر بیشتری بر خروجی‌های دنباله دارند.

یک مدل دنباله به دنباله، مانند ترجمه ماشینی را در نظر بگیرید. این مدل توسط دو شبکه بازگشتی پیاده‌سازی می‌شود، که در آن یک شبکه (**رمزگذار**) دنباله ورودی را به حالت پنهان تبدیل می‌کند و شبکه دیگر (**رمزگشا**) این حالت پنهان را به نتیجه ترجمه‌شده باز می‌گرداند. مشکل این روش این است که حالت نهایی شبکه به سختی می‌تواند ابتدای جمله را به خاطر بسپارد، که این موضوع باعث کاهش کیفیت مدل در جملات طولانی می‌شود.

**مکانیزم‌های توجه** راهی برای وزن‌دهی تأثیر متنی هر بردار ورودی بر هر پیش‌بینی خروجی RNN فراهم می‌کنند. این مکانیزم با ایجاد میان‌برهایی بین حالت‌های میانی RNN ورودی و RNN خروجی پیاده‌سازی می‌شود. به این ترتیب، هنگام تولید نماد خروجی $y_t$، تمام حالت‌های پنهان ورودی $h_i$ را با ضرایب وزنی مختلف $\alpha_{t,i}$ در نظر می‌گیریم.

![تصویری از مدل رمزگذار/رمزگشا با لایه توجه افزایشی](../../../../../lessons/5-NLP/18-Transformers/images/encoder-decoder-attention.png)
*مدل رمزگذار-رمزگشا با مکانیزم توجه افزایشی در [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf)، نقل شده از [این پست وبلاگ](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

ماتریس توجه $\{\alpha_{i,j}\}$ نشان‌دهنده درجه‌ای است که کلمات خاص ورودی در تولید یک کلمه خاص در دنباله خروجی نقش دارند. در زیر نمونه‌ای از چنین ماتریسی آورده شده است:

![تصویری از یک هم‌ترازی نمونه که توسط RNNsearch-50 پیدا شده است، گرفته شده از Bahdanau - arviz.org](../../../../../lessons/5-NLP/18-Transformers/images/bahdanau-fig3.png)

*تصویر گرفته شده از [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (شکل ۳)*

مکانیزم‌های توجه مسئول بخش عمده‌ای از وضعیت فعلی یا نزدیک به وضعیت فعلی پیشرفته در پردازش زبان طبیعی هستند. با این حال، افزودن توجه به طور قابل توجهی تعداد پارامترهای مدل را افزایش می‌دهد که منجر به مشکلات مقیاس‌پذیری در RNN‌ها می‌شود. یکی از محدودیت‌های کلیدی مقیاس‌پذیری RNN‌ها این است که ماهیت بازگشتی مدل‌ها باعث می‌شود آموزش دسته‌ای و موازی‌سازی چالش‌برانگیز باشد. در RNN، هر عنصر از یک دنباله باید به ترتیب پردازش شود، که به این معنی است که نمی‌توان آن را به راحتی موازی‌سازی کرد.

پذیرش مکانیزم‌های توجه همراه با این محدودیت منجر به ایجاد مدل‌های ترانسفورمر شد که اکنون به عنوان وضعیت پیشرفته شناخته می‌شوند و امروزه از BERT تا OpenGPT3 استفاده می‌کنیم.

## مدل‌های ترانسفورمر

به جای انتقال زمینه هر پیش‌بینی قبلی به مرحله ارزیابی بعدی، **مدل‌های ترانسفورمر** از **رمزگذاری موقعیتی** و توجه برای درک زمینه یک ورودی خاص در یک پنجره متن ارائه‌شده استفاده می‌کنند. تصویر زیر نشان می‌دهد که چگونه رمزگذاری موقعیتی همراه با توجه می‌تواند زمینه را در یک پنجره مشخص درک کند.

![GIF متحرک نشان‌دهنده نحوه انجام ارزیابی‌ها در مدل‌های ترانسفورمر.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

از آنجا که هر موقعیت ورودی به طور مستقل به هر موقعیت خروجی نگاشت می‌شود، ترانسفورمرها بهتر از RNN‌ها موازی‌سازی می‌شوند، که امکان ایجاد مدل‌های زبانی بسیار بزرگ‌تر و بیان‌گرتر را فراهم می‌کند. هر سر توجه می‌تواند برای یادگیری روابط مختلف بین کلمات استفاده شود که وظایف پردازش زبان طبیعی را بهبود می‌بخشد.

**BERT** (نمایش‌های رمزگذار دوطرفه از ترانسفورمرها) یک شبکه ترانسفورمر چندلایه بسیار بزرگ است که دارای ۱۲ لایه برای *BERT-base* و ۲۴ لایه برای *BERT-large* است. این مدل ابتدا بر روی مجموعه داده‌های متنی بزرگ (ویکی‌پدیا + کتاب‌ها) با استفاده از آموزش بدون نظارت (پیش‌بینی کلمات ماسک‌شده در یک جمله) پیش‌آموزش داده می‌شود. در طول پیش‌آموزش، مدل سطح قابل توجهی از درک زبان را جذب می‌کند که سپس می‌توان آن را با مجموعه داده‌های دیگر از طریق تنظیم دقیق استفاده کرد. این فرآیند **یادگیری انتقالی** نامیده می‌شود.

![تصویر از http://jalammar.github.io/illustrated-bert/](../../../../../lessons/5-NLP/18-Transformers/images/jalammarBERT-language-modeling-masked-lm.png)

مدل‌های ترانسفورمر دارای انواع مختلفی هستند، از جمله BERT، DistilBERT، BigBird، OpenGPT3 و موارد دیگر که می‌توان آنها را تنظیم دقیق کرد. بسته [HuggingFace](https://github.com/huggingface/) مخزنی برای آموزش بسیاری از این معماری‌ها با PyTorch فراهم می‌کند.

## استفاده از BERT برای طبقه‌بندی متن

بیایید ببینیم چگونه می‌توانیم از مدل پیش‌آموزش داده‌شده BERT برای حل وظیفه سنتی خود: طبقه‌بندی دنباله استفاده کنیم. ما مجموعه داده اصلی AG News خود را طبقه‌بندی خواهیم کرد.

ابتدا، کتابخانه HuggingFace و مجموعه داده خود را بارگذاری می‌کنیم:


In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


از آنجا که ما قصد داریم از مدل از پیش آموزش‌دیده‌ی BERT استفاده کنیم، نیاز داریم که از یک توکنایزر خاص استفاده کنیم. ابتدا، توکنایزری که با مدل از پیش آموزش‌دیده‌ی BERT مرتبط است را بارگذاری می‌کنیم.

کتابخانه‌ی HuggingFace شامل یک مخزن از مدل‌های از پیش آموزش‌دیده است که می‌توانید تنها با مشخص کردن نام آن‌ها به عنوان آرگومان در توابع `from_pretrained` از آن‌ها استفاده کنید. تمام فایل‌های باینری مورد نیاز برای مدل به صورت خودکار دانلود خواهند شد.

با این حال، در برخی مواقع ممکن است نیاز داشته باشید مدل‌های خودتان را بارگذاری کنید. در این صورت می‌توانید دایرکتوری‌ای را مشخص کنید که شامل تمام فایل‌های مرتبط باشد، از جمله پارامترهای توکنایزر، فایل `config.json` با پارامترهای مدل، وزن‌های باینری و غیره.


In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

شیء `tokenizer` شامل تابع `encode` است که می‌توان آن را مستقیماً برای کدگذاری متن استفاده کرد:


In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

سپس، بیایید تکرارگرهایی ایجاد کنیم که در طول آموزش برای دسترسی به داده‌ها استفاده خواهیم کرد. از آنجا که BERT از تابع رمزگذاری خود استفاده می‌کند، نیاز داریم یک تابع پرکننده مشابه `padify` که قبلاً تعریف کرده‌ایم، تعریف کنیم:


In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

در مورد ما، از مدل از پیش آموزش‌دیده‌ی BERT به نام `bert-base-uncased` استفاده خواهیم کرد. بیایید مدل را با استفاده از بسته‌ی `BertForSequenceClassfication` بارگذاری کنیم. این کار تضمین می‌کند که مدل ما از قبل معماری مورد نیاز برای طبقه‌بندی، شامل طبقه‌بند نهایی را دارد. شما پیامی هشداردهنده خواهید دید که بیان می‌کند وزن‌های طبقه‌بند نهایی مقداردهی اولیه نشده‌اند و مدل نیاز به پیش‌آموزش دارد - این کاملاً مشکلی ندارد، زیرا دقیقاً همان کاری است که قصد انجام آن را داریم!


In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

حالا آماده شروع آموزش هستیم! از آنجایی که BERT از قبل آموزش دیده است، می‌خواهیم با نرخ یادگیری نسبتاً کوچکی شروع کنیم تا وزن‌های اولیه تخریب نشوند.

تمام کارهای سخت توسط مدل `BertForSequenceClassification` انجام می‌شود. وقتی مدل را روی داده‌های آموزشی فراخوانی می‌کنیم، هم مقدار زیان (loss) و هم خروجی شبکه را برای مینی‌بچ ورودی برمی‌گرداند. ما از زیان برای بهینه‌سازی پارامترها استفاده می‌کنیم (`loss.backward()` مرحله بازگشتی را انجام می‌دهد) و از `out` برای محاسبه دقت آموزش با مقایسه برچسب‌های به‌دست‌آمده `labs` (که با استفاده از `argmax` محاسبه می‌شوند) با برچسب‌های مورد انتظار `labels` استفاده می‌کنیم.

برای کنترل فرآیند، زیان و دقت را در چندین تکرار جمع‌آوری می‌کنیم و آن‌ها را هر `report_freq` چرخه آموزشی چاپ می‌کنیم.

این آموزش احتمالاً زمان زیادی طول خواهد کشید، بنابراین تعداد تکرارها را محدود می‌کنیم.


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


شما می‌توانید مشاهده کنید (به‌ویژه اگر تعداد تکرارها را افزایش دهید و زمان کافی اختصاص دهید) که طبقه‌بندی با BERT دقت بسیار خوبی به ما می‌دهد! این به این دلیل است که BERT ساختار زبان را به‌خوبی درک می‌کند و ما فقط نیاز داریم طبقه‌بند نهایی را تنظیم کنیم. با این حال، از آنجا که BERT یک مدل بزرگ است، کل فرآیند آموزش زمان زیادی می‌برد و به قدرت محاسباتی قابل توجهی نیاز دارد! (GPU، و ترجیحاً بیش از یک عدد).

> **توجه:** در مثال ما، از یکی از کوچک‌ترین مدل‌های پیش‌آموزش‌دیده BERT استفاده کرده‌ایم. مدل‌های بزرگ‌تری وجود دارند که احتمالاً نتایج بهتری ارائه می‌دهند.


## ارزیابی عملکرد مدل

اکنون می‌توانیم عملکرد مدل خود را بر روی مجموعه داده آزمایشی ارزیابی کنیم. حلقه ارزیابی بسیار شبیه به حلقه آموزش است، اما نباید فراموش کنیم که مدل را با فراخوانی `model.eval()` به حالت ارزیابی تغییر دهیم.


In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## نکات کلیدی

در این بخش، مشاهده کردیم که چقدر آسان است یک مدل زبانی از پیش آموزش‌دیده از کتابخانه **transformers** را گرفته و آن را برای وظیفه طبقه‌بندی متن خود تطبیق دهیم. به همین ترتیب، مدل‌های BERT می‌توانند برای استخراج موجودیت‌ها، پاسخ به سوالات و سایر وظایف پردازش زبان طبیعی استفاده شوند.

مدل‌های ترانسفورمر نمایانگر پیشرفته‌ترین وضعیت در پردازش زبان طبیعی هستند و در بیشتر موارد باید اولین راه‌حلی باشند که هنگام پیاده‌سازی راه‌حل‌های سفارشی پردازش زبان طبیعی با آن شروع به آزمایش می‌کنید. با این حال، درک اصول پایه‌ای شبکه‌های عصبی بازگشتی که در این بخش مورد بحث قرار گرفتند، بسیار مهم است اگر بخواهید مدل‌های عصبی پیشرفته بسازید.



---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما تلاش می‌کنیم دقت را حفظ کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطاها یا نادرستی‌ها باشند. سند اصلی به زبان اصلی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، توصیه می‌شود از ترجمه حرفه‌ای انسانی استفاده کنید. ما مسئولیتی در قبال سوء تفاهم‌ها یا تفسیرهای نادرست ناشی از استفاده از این ترجمه نداریم.
